In [2]:
import os
import numpy as np
import pandas as pd

def process_data(directory):
    # Fetch all CSV files in the directory
    file_path = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.csv')]
    
    # Initialize a list to store data from all CSV files
    df_list = []

    # Loop through all CSV files
    for file in file_path:
        try:
            # Read the CSV file without skipping any rows
            df = pd.read_csv(file, sep='\t', header=None, index_col=False, comment='#', on_bad_lines='warn')
            df_list.append(df)
        except pd.errors.ParserError as e:
            print(f"Error reading {file}: {e}")

    # Concatenate all DataFrames
    combined_df = pd.concat(df_list, ignore_index=True)

    # Extract ADC data from the first column
    adc_data = combined_df.iloc[:, 0]

    # Convert ADC data to a numpy float array
    adc_array = pd.to_numeric(adc_data, errors='coerce').fillna(0).to_numpy()

    return adc_array

def compute_fft_and_save(adc_array, output_path):
    # Apply a Hamming window to the data
    window_ham = np.hamming(len(adc_array))
    window_ham_data = adc_array * window_ham

    # Perform FFT on the windowed data
    fft_result = np.fft.fft(window_ham_data)

    # Calculate frequency and magnitude
    sampling_rate = 16000
    n = len(adc_array)
    freq = np.fft.fftfreq(n, d=1 / sampling_rate)
    magnitude = np.abs(fft_result)

    # Stack frequency and magnitude for saving as .npy
    fft_data = np.stack((freq, magnitude), axis=-1)

    # Save FFT data as an .npy file
    np.save(output_path, fft_data)

# Directory paths for human presence and absence data
human_present_dir = "C:/Users/madhu/Desktop/individual_project/training_data/new_sensor_train_data/with_human"
human_absent_dir = "C:/Users/madhu/Desktop/individual_project/training_data/new_sensor_train_data/without_human"  # Adjust as necessary

# Output paths for FFT results
output_dir = "C:/Users/madhu/Desktop/individual_project/outputs/output_train/fft_output_training_data"
human_present_output = os.path.join(output_dir, "fft_human_present.npy")
human_absent_output = os.path.join(output_dir, "fft_human_absent.npy")

# Process ADC data for human presence and absence
adc_array_human_present = process_data(human_present_dir)
adc_array_human_absent = process_data(human_absent_dir)

# Compute FFT and save for both conditions
compute_fft_and_save(adc_array_human_present, human_present_output)
compute_fft_and_save(adc_array_human_absent, human_absent_output)

print("FFT computation completed and results saved as .npy files.")


FFT computation completed and results saved as .npy files.
